## MIPHA test on dummy data

The purpose of this notebook is to test functions of the MIPHA framework on dummy data with placeholder implementations. This will allow for faster, targeted testing.

In [ ]:
import sys
from importlib import reload

import tests.test_dummy_data.dummy_data_manager as data
import tests.test_dummy_data.dummy_implementation as impl  # necessary for pickle to work, cannot use from ... import *
from src.mipha.framework import MiphaPredictor, DataSource

In [ ]:
reload(sys.modules['src.mipha.framework'])
reload(sys.modules['tests.test_dummy_data.dummy_data_manager'])
reload(sys.modules['tests.test_dummy_data.dummy_implementation'])

### Framework implementation



In [ ]:
x_train, y_train, x_test, y_test = data.load_dummy_data(
    len_x_train=10,
    len_y_train=10,
    len_x_test=5,
    len_y_test=5,
    num_data_sources=3
)

In [ ]:
feature_extractor_1 = impl.DummyFeatureExtractor(component_name="FE1", managed_data_types=["TypeA", "TypeB"])
feature_extractor_2 = impl.DummyFeatureExtractor(component_name="FE2", managed_data_types=["TypeA"])
feature_extractor_3 = impl.DummyFeatureExtractor(component_name="FE3", managed_data_types=["TypeB"])

aggregator = impl.DummyAggregator()
model = impl.DummyMachineLearningModel()
evaluator = impl.DummyEvaluator()

In [ ]:
mipha = MiphaPredictor(
    feature_extractors=[feature_extractor_1, feature_extractor_2, feature_extractor_3],
    aggregator=aggregator,
    model=model,
    evaluator=evaluator,
)

In [ ]:
print("fit")

dummy_data_source_1 = DataSource(
    data_type="TypeA",
    data=x_train[0],
    name="dummy_data_source_1",
)

dummy_data_source_2 = DataSource(
    data_type="TypeB",
    data=x_train[1],
    name="dummy_data_source_2",
)

dummy_data_source_3 = DataSource(
    data_type="TypeA",
    data=x_train[2],
    name="dummy_data_source_3",
)

data_sources = [dummy_data_source_1, dummy_data_source_2, dummy_data_source_3]

mipha.fit(
    data_sources=data_sources,
    train_labels=y_train
)

print("predict")
mipha.predict(
    data_sources=data_sources,
    train_labels=y_train
)

print("evaluate")
mipha.evaluate(
    data_sources=data_sources,
    train_labels=y_train,
    test_labels=y_test,
)

In [ ]:
# Saving and loading the model
archive_path = "out/test.zip"
mipha.save(archive_path)
mipha_loaded = MiphaPredictor.load(archive_path)

In [ ]:
# Fitting with precomputed features
mipha.fit(data_sources=data_sources, train_labels=y_train, precomputed_features=mipha.last_computed_features)